# Tool Use

The `AgentChat` api provides a `ToolUseAssistantAgent` with presets for adding tools that the agent can call as part of it's response. 

:::{note}

The example presented here is a work in progress 🚧. Also, tool uses here assumed the `model_client` used by the agent supports tool calling. 
::: 

In [1]:
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.teams import EVENT_LOGGER_NAME, RoundRobinGroupChat, StopMessageTermination
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

In AgentChat, a Tool is a function wrapped in the `FunctionTool` class exported from `autogen_core.components.tools`.   

In [2]:
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."


get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

Finally, agents that use tools are defined in the following manner. 

-  An agent is instantiated based on the  `ToolUseAssistantAgent` class in AgentChat. The agent is aware of the tools it can use by passing a `tools_schema` attribute to the class, which is passed to the `model_client` when the agent generates a response.
-  An agent Team is defined that takes a list of `tools`.  Effectively, the `ToolUseAssistantAgent` can generate messages that call tools, and the team is responsible executing those tool calls and returning the results.

In [3]:
assistant = ToolUseAssistantAgent(
    "Weather_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    registered_tools=[get_weather_tool],
)
team = RoundRobinGroupChat([assistant])
result = await team.run("What's the weather in New York?", termination_condition=StopMessageTermination())


--------------------------------------------------------------------------- 
[2024-10-08T20:34:31.935149]:

What's the weather in New York?


--------------------------------------------------------------------------- 
[2024-10-08T20:34:33.080494], Weather_Assistant:

The weather in New York is 72 degrees and sunny. 

TERMINATE

## Using Langchain Tools 

AutoGen also provides direct support for tools from LangChain via the `autogen_ext`  package.



In [6]:
# pip install langchain, langchain-community, wikipedia , autogen-ext

import wikipedia
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(wiki_client=wikipedia, top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

langchain_wikipedia_tool = LangChainToolAdapter(tool)

In [7]:
wikipedia_assistant = ToolUseAssistantAgent(
    "WikiPedia_Assistant",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    registered_tools=[langchain_wikipedia_tool],
)
team = RoundRobinGroupChat([wikipedia_assistant])
result = await team.run(
    "Who was the first president of the United States?", termination_condition=StopMessageTermination()
)


--------------------------------------------------------------------------- 
[2024-10-08T20:44:08.218758]:

Who was the first president of the United States?
--------------------------------------------------------------------------- 
[2024-10-08T20:44:11.240067], WikiPedia_Assistant:

The first president of the United States was George Washington, who served from April 30, 1789, to March 4, 1797. 

TERMINATE